In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
import pickle
import math
import seaborn
import matplotlib.pyplot as plt

In [2]:
def cleanUp(df):
    df.drop(columns=["Team_abbrev", "Opponent_abbrev", "vis_team", "home_team", "Roof", "Surface"], inplace = True)

    df.drop(columns=["Vegas_Favorite"], inplace=True)

    df.drop(columns=["game_date"], inplace=True)

    df.drop(columns=["Vegas_Line", "Over_Under"], inplace=True)

def normalize_data(df):
    df = (df - df.mean())/df.std()
    df.dropna(axis=1, how='all', inplace=True)
    return df

def split_data(df, string):
    Y = np.asarray(df[string])
    df.drop(columns = [string], inplace = True)
    X = np.asarray(df).astype(np.float32)
    X = X.reshape(X.shape[0], -1)
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=.2)
    return X,Y, Xtrain, Xtest, ytrain, ytest

def train(Xtrain, ytrain):
    model_forest = RandomForestRegressor()
    forest=model_forest.fit(Xtrain, ytrain)
    return forest

def analyze(df, string,model, Xtest, ytest):
    ypred = model.predict(Xtest)
    mn = df[string].mean()
    st = df[string].std()
    ypred  = (ypred * st) + mn
    ytest = (ytest * st) + mn
    ypred = np.around(ypred)
    ytest = np.around(ytest)
    return ypred, ytest

def save_model(model, name):
    filename = 'model' + name + '.sav'
    pickle.dump(model, open(filename, 'wb'))

def create_model(current_st):
    df_nfl_players_stats = pd.read_csv("C:/Users/carlo/Downloads/nfl_pass_rush_receive_raw_data.csv")
    cleanUp(df_nfl_players_stats)
    norm = normalize_data(df_nfl_players_stats)
    X,Y, Xtrain, Xtest, ytrain, ytest = split_data(norm, current_st)
    model = train(Xtrain, ytrain)
    ypred, ytest = analyze(df_nfl_players_stats, current_st, model, Xtest, ytest)
    save_model(model, current_st) 

In [6]:
df_nfl_players_stats = pd.read_csv("C:/Users/carlo/Downloads/nfl_pass_rush_receive_raw_data.csv")

C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
current_stadistic = "rush_yds"
cleanUp(df_nfl_players_stats)
norm = normalize_data(df_nfl_players_stats)
X,Y, Xtrain, Xtest, ytrain, ytest = split_data(norm, current_stadistic)
model = train(Xtrain, ytrain)
ypred, ytest = analyze(df_nfl_players_stats, current_stadistic, model, Xtest, ytest)
save_model(model, current_stadistic) 

In [11]:
ytest.max()

211.0

In [12]:
ytest.min()

-20.0

In [13]:
ytest.mean()

9.832330827067668

In [9]:
ypred

array([ 0.,  0.,  0., ..., 16., 19.,  0.])

In [10]:
cnf_matrix = confusion_matrix(ytest,ypred)
print(cnf_matrix)
print(classification_report(ytest,ypred))

[[0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]
              precision    recall  f1-score   support

       -20.0       0.00      0.00      0.00         1
       -15.0       0.00      0.00      0.00         0
       -13.0       0.00      0.00      0.00         1
       -12.0       0.00      0.00      0.00         1
       -11.0       0.20      1.00      0.33         1
       -10.0       1.00      0.67      0.80         3
        -8.0       0.00      0.00      0.00         1
        -7.0       1.00      0.25      0.40         4
        -6.0       0.33      1.00      0.50         1
        -5.0       0.80      1.00      0.89         4
        -4.0       1.00      0.88      0.93         8
        -3.0       1.00      1.00      1.00        25
        -2.0       0.95      0.95      0.95        20
        -1.0       0.98      1.00      0.99        49
         0.0       1.00      1.00      1.00      3519
         1.0   

C:\Users\carlo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [ ]:
df_nfl_players_stats

In [ ]:
df_nfl_players_stats[df_nfl_players_stats["pos"]=="QB"]

In [ ]:
model_forest = RandomForestRegressor()
forest=model_forest.fit(Xtrain, ytrain)

ypred = forest.predict(Xtest)
ypred = np.around(ypred)
cnf_matrix = confusion_matrix(ytest,ypred)
print(cnf_matrix)
print(classification_report(ytest,ypred))

seaborn.heatmap(cnf_matrix, annot=True)

## Only QB DATA TRAINING

In [ ]:
df_nfl_players_stats = pd.read_csv("C:/Users/carlo/Downloads/nfl_pass_rush_receive_raw_data.csv")

In [ ]:
df_nfl_players_stats.drop(columns=["Team_abbrev", "Opponent_abbrev", "vis_team", "home_team", "Roof", "Surface"], inplace = True)

df_nfl_players_stats.drop(columns=["Vegas_Favorite"], inplace=True)

df_nfl_players_stats.drop(columns=["game_date"], inplace=True)

df_nfl_players_stats.drop(columns=["Vegas_Line", "Over_Under"], inplace=True)

In [ ]:
df_qb = df_nfl_players_stats[df_nfl_players_stats["pos"]=="QB"]

In [ ]:
df_qb

In [ ]:
df1 = pd.read_csv("C:/Users/carlo/Downloads/nfl_pass_rush_receive_raw_data.csv")

In [ ]:
cleanUp(df1)

In [ ]:
norm = normalize_data(df1)

In [ ]:
Y = np.asarray(df_qb["pass_cmp"])
df_qb.drop(columns=["pass_cmp"], inplace=True)

In [ ]:
df_qb

In [ ]:
X = np.asarray(df_qb.iloc[:,5:]).astype(np.float32)
X = X.reshape(X.shape[0], -1)

In [ ]:
X,Y =split_data(norm, "pass_cmp")

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=.2)

In [ ]:
model_forest = RandomForestRegressor()
forest=model_forest.fit(Xtrain, ytrain)

In [ ]:
ytest

In [ ]:
ypred

In [ ]:
ypred, ytest = analyze(df_nfl_players_stats, "pass_cmp", forest, Xtest, ytest)

In [ ]:
ypred

In [ ]:
ytest

In [ ]:
cnf_matrix = confusion_matrix(ytest,ypred)
print(cnf_matrix)
print(classification_report(ytest,ypred))

In [ ]:
save_model(forest, "pass_cmp")

In [ ]:
ytest

## Receiver Stats Training 
### WR / TE / RB

In [ ]:
df_nfl_players_stats = pd.read_csv("C:/Users/carlo/Downloads/nfl_pass_rush_receive_raw_data.csv")
df_nfl_players_stats.drop(columns=["Team_abbrev", "Opponent_abbrev", "vis_team", "home_team", "Roof", "Surface"], inplace = True)
df_nfl_players_stats.drop(columns=["Vegas_Favorite"], inplace=True)
df_nfl_players_stats.drop(columns=["game_date"], inplace=True)
df_nfl_players_stats.drop(columns=["Vegas_Line", "Over_Under"], inplace=True)

In [ ]:
df_receiver = df_nfl_players_stats.copy()
df_receiver= df_receiver[df_receiver["pos"].isin(["WR", "TE", "RB"])]

In [ ]:
df_receiver

In [ ]:
Y = np.asarray(df_receiver["rec"])
df_receiver.drop(columns=["rec"], inplace=True)

X = np.asarray(df_receiver.iloc[:,5:]).astype(np.float32)
X = X.reshape(X.shape[0], -1)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=.2)

In [ ]:
model_receiver = RandomForestRegressor()
forest_receiver=model_receiver.fit(Xtrain, ytrain)

ypred = forest_receiver.predict(Xtest)
ypred = np.around(ypred)
cnf_matrix = confusion_matrix(ytest,ypred)
print(cnf_matrix)
print(classification_report(ytest,ypred))

seaborn.heatmap(cnf_matrix, annot=True)

In [ ]:
df_nfl_players_stats = pd.read_csv("C:/Users/carlo/Downloads/nfl_pass_rush_receive_raw_data.csv")
df_nfl_players_stats.drop(columns=["Team_abbrev", "Opponent_abbrev", "vis_team", "home_team", "Roof", "Surface"], inplace = True)
df_nfl_players_stats.drop(columns=["Vegas_Favorite"], inplace=True)
df_nfl_players_stats.drop(columns=["game_date"], inplace=True)
df_nfl_players_stats.drop(columns=["Vegas_Line", "Over_Under"], inplace=True)
df_receiver_all = df_nfl_players_stats.copy()

In [ ]:
Y = np.asarray(df_receiver_all["rec"])
df_receiver_all.drop(columns=["rec"], inplace=True)

X = np.asarray(df_receiver_all.iloc[:,5:]).astype(np.float32)
X = X.reshape(X.shape[0], -1)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=.2)

In [ ]:
model_receiver_all = RandomForestRegressor()
forest_receiver_all=model_receiver_all.fit(Xtrain, ytrain)

ypred = forest_receiver_all.predict(Xtest)
ypred = np.around(ypred)
cnf_matrix = confusion_matrix(ytest,ypred)
print(cnf_matrix)
print(classification_report(ytest,ypred))

In [ ]:
importances = forest_receiver_all.feature_importances_

In [ ]:
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)

In [ ]:
index = list(df_receiver_all.columns)
del index[:5]

In [ ]:
print(index)

In [ ]:
import matplotlib.pyplot as plt
forest_importances = pd.Series(importances, index=index)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
importances

In [ ]:
df_nfl_players_stats